# compare hichip

song
inoue?


In [1]:
import pandas as pd
import pybedtools
import os, glob
from collections import defaultdict

In [2]:
save_dir = '../data/processed/compare_external/song_hichip/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [59]:
def get_stats(feature_dict1, feature_dict2, save_prefix, 
              save_bed = False, # save bed A
              save_dir=save_dir):
    stats_dict = {}
    stats_idx = 0
    for my_sampleA, my_bedA in feature_dict1.items():
        num_anchorsA = len(my_bedA)
        for my_sampleB, my_bedB in feature_dict2.items():
            num_anchorsB = len(my_bedB)
            if save_bed:
                new_name = '__'.join([my_sampleA, my_sampleB,'intersect'])
                my_bedA.intersect(my_bedB,wa=True,f=0.25).saveas(os.path.join(save_dir, new_name+'.bed'))
            stats_dict[stats_idx] = {'sampleA':my_sampleA,
                                    'sampleB':my_sampleB,
                                    'num_anchorsA':num_anchorsA,
                                    'num_anchorsB':num_anchorsB}
            stats_dict[stats_idx].update(my_bedA.jaccard(b=my_bedB,f=0.25))
            stats_idx+=1

    stats_df = pd.DataFrame.from_dict(stats_dict,orient='index')
    stats_df['perc_anchorsA_captured'] = stats_df.n_intersections/stats_df.num_anchorsA
    stats_df['perc_anchorsB_captured'] = stats_df.n_intersections/stats_df.num_anchorsB
    stats_df.to_csv(os.path.join(save_dir, 'stats_'+save_prefix+'_comp.csv'))   
    return stats_df



In [4]:
my_anchor_dir = '../data/interim/merged/anchors_bed_sort//'
my_anchor_files = glob.glob(os.path.join(my_anchor_dir, '*'))
my_anchor_files_named = {}
for file in my_anchor_files:
    sample = os.path.basename(file).split('_sort')[0]
    my_anchor_files_named[sample] = pybedtools.BedTool(file)
    print(sample, len(my_anchor_files_named[sample]))
my_anchor_files, my_anchor_files_named.keys() 

SL_D2 460761
SLC_D2 438625
H9_D28 345113
H9_D10 230431
H9_D2 461684
Astrocytes 322550
SL_D0 297076
SLC_D0 348166
H9_D0 306851


(['../data/interim/merged/anchors_bed_sort/SL_D2_sort.bed',
  '../data/interim/merged/anchors_bed_sort/SLC_D2_sort.bed',
  '../data/interim/merged/anchors_bed_sort/H9_D28_sort.bed',
  '../data/interim/merged/anchors_bed_sort/H9_D10_sort.bed',
  '../data/interim/merged/anchors_bed_sort/H9_D2_sort.bed',
  '../data/interim/merged/anchors_bed_sort/Astrocytes_sort.bed',
  '../data/interim/merged/anchors_bed_sort/SL_D0_sort.bed',
  '../data/interim/merged/anchors_bed_sort/SLC_D0_sort.bed',
  '../data/interim/merged/anchors_bed_sort/H9_D0_sort.bed'],
 dict_keys(['SL_D2', 'SLC_D2', 'H9_D28', 'H9_D10', 'H9_D2', 'Astrocytes', 'SL_D0', 'SLC_D0', 'H9_D0']))

In [5]:
my_loop_files = glob.glob('../data/interim/merged/loops/*')
my_loop_files_named = {}
for file in my_loop_files:
    sample = os.path.basename(file).split('.')[0]
    my_loop_files_named[sample] = pd.read_csv(file,index_col=0)
    print(sample, my_loop_files_named[sample].shape)


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


SL_D2 (5324429, 3)
Astrocytes (1839205, 3)
SLC_D2 (4282155, 3)
H9_D0 (1318716, 3)
SL_D0 (1187599, 3)
SLC_D0 (2659082, 3)
H9_D10 (712533, 3)
H9_D2 (6720106, 3)
H9_D28 (1256959, 3)


In [26]:
my_loop_files_named['SL_D0']

,source,target,count
43,chr10_1000000_1005000,chr10_1020000_1025000,43
48,chr10_1000000_1005000,chr10_1055000_1060000,3
49,chr10_1000000_1005000,chr10_1060000_1065000,3
51,chr10_1000000_1005000,chr10_1080000_1085000,1
109,chr10_100000_105000,chr10_1020000_1025000,1
...,...,...,...
14056146,chrX_99940000_99945000,chrX_100590000_100595000,1
14056163,chrX_99970000_99975000,chrX_100060000_100065000,1
14056186,chrX_99995000_100000000,chrX_100060000_100065000,1
14056189,chrX_99995000_100000000,chrX_100170000_100175000,1


In [58]:
song_pcdir = '/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/song_pchic'
song_loop_files = glob.glob(os.path.join(song_pcdir, '*txt'))

song_anchor_files_named = {}

song_loop_files_named = {}
for file in song_loop_files:
    sample = os.path.basename(file).split('.')[0].split('_')[1]

    df = pd.read_table(file,names=['source','target','count'])
    df[['chrx','startx','stopx']]  = df.source.str.split(',',expand=True)
    df[['chry','starty','stopy']]  = df.target.str.split(',',expand=True)
    df['startx_round'] = (round(df['startx'].map(int) / 5000) * 5000).map(int)
    df['stopx_round_prelim'] = (round(df['stopx'].map(int) / 5000) * 5000)
    df['stopx_round'] = df['startx_round']+5000
    df['stopx_round'] = df[['stopx_round_prelim','stopx_round']].max(axis=1).map(int)
    df['starty_round'] = (round(df['starty'].map(int) / 5000) * 5000).map(int)
    df['stopy_round_prelim'] = (round(df['stopy'].map(int) / 5000) * 5000)
    df['stopy_round'] = df['starty_round']+5000
    df['stopy_round'] = df[['stopy_round_prelim','stopy_round']].max(axis=1).map(int)
    df['source_arr'] = df.apply(lambda row: [row.chrx+'_'+str(start) + '_'+ str(start+5000) for start in list(range(row.startx_round, row.stopx_round, 5000))],axis=1)
    df['target_arr'] = df.apply(lambda row: [row.chry+'_'+str(start) + '_'+ str(start+5000) for start in list(range(row.starty_round, row.stopy_round, 5000))],axis=1)

    df = df[['source_arr','target_arr','count']].explode('source_arr').explode('target_arr')
    df.columns = ['source','target', 'count']
    
    df.to_csv(os.path.join(song_pcdir, sample+'.loops.csv'))
                            
    anc_df = pd.DataFrame(pd.concat([df.source , df.target]).unique(),columns=['name'])
    anc_df[['chr','start','stop']] = anc_df.name.str.split('_',expand=True)
    anc_df = anc_df[['chr', 'start', 'stop', 'name'   ]]
    anc_bed = pybedtools.BedTool.from_dataframe(anc_df).sort()
    anc_bed.saveas(os.path.join(song_pcdir, sample+'_sort.bed'))
                            
                            
    song_loop_files_named[sample] = df
    song_anchor_files_named[sample] = anc_bed
                            
    print(sample, song_loop_files_named[sample].shape, len(anc_bed))

motor (66978, 3) 45607
cortical (73890, 3) 51963
astrocyte (84087, 3) 52724
hippocampal (108156, 3) 71024


In [60]:
%%time
# compare anchors - similar to atac peaks
stats_df_song_my = get_stats(my_anchor_files_named, song_anchor_files_named, 
                             save_bed=True, 
                             save_prefix='my_song')
    

KeyError: 'perc_peaksA_captured'

In [62]:
stats_df_song_my.sort_values('perc_anchorsB_captured',ascending=True)


,sampleA,sampleB,num_anchorsA,num_anchorsB,intersection,union,jaccard,n_intersections,perc_anchorsA_captured,perc_anchorsB_captured
8,H9_D28,motor,345113,45607,67950000,1885650000,0.036035,6281,0.018200,0.137720
10,H9_D28,astrocyte,345113,52724,91220000,1897965000,0.048062,7666,0.022213,0.145399
9,H9_D28,cortical,345113,51963,82590000,1902790000,0.043405,7766,0.022503,0.149452
11,H9_D28,hippocampal,345113,71024,114310000,1966375000,0.058132,11008,0.031897,0.154990
19,H9_D2,hippocampal,461684,71024,134465000,2529075000,0.053168,13247,0.028693,0.186514
17,H9_D2,cortical,461684,51963,99320000,2468915000,0.040228,9831,0.021294,0.189192
3,SL_D2,hippocampal,460761,71024,136205000,2522720000,0.053991,13663,0.029653,0.192372
18,H9_D2,astrocyte,461684,52724,111385000,2460655000,0.045266,10211,0.022117,0.193669
16,H9_D2,motor,461684,45607,88540000,2447915000,0.036170,8867,0.019206,0.194422
1,SL_D2,cortical,460761,51963,100425000,2463195000,0.040770,10138,0.022003,0.195100


In [71]:
# compare loops just report # loops that overlap
stats_dict = {}
stats_idx = 0
for sampleA, loop_dfA in my_loop_files_named.items():
    loop_dfA_arr = set(loop_dfA.source +'::' + loop_dfA.target)
    for sampleB, loop_dfB in song_loop_files_named.items():
        loop_dfB = loop_dfB[['source','target']].drop_duplicates()
        loop_dfB_arr = set(loop_dfB.source +'::' + loop_dfB.target)
        num_loopA = len(loop_dfA_arr)
        num_loopB = len(loop_dfB_arr)
        num_intersection = len(loop_dfA_arr.intersection(loop_dfB_arr))
        num_union = len(loop_dfA_arr.union(loop_dfB_arr))
        
        stats_dict[stats_idx] = {'sampleA':sampleA,
                                'sampleB':sampleB,
                                'num_loopA':num_loopA,
                                'num_loopB':num_loopB,
                                'num_intersection_loops':num_intersection,
                                'num_union':num_union}
        stats_idx+=1

In [73]:
stats_df_song_loop = pd.DataFrame.from_dict(stats_dict,orient='index')
stats_df_song_loop['perc_loopsA_captured'] = stats_df_song_loop.num_intersection_loops/stats_df_song_loop.num_loopA
stats_df_song_loop['perc_loopsB_captured'] = stats_df_song_loop.num_intersection_loops/stats_df_song_loop.num_loopB

stats_df_song_loop.to_csv(os.path.join(save_dir, 'stats_'+'my_song_loop'+'_comp.csv'))   


In [75]:
stats_df_song_loop.sort_values('')

,sampleA,sampleB,num_loopA,num_loopB,num_intersection_loops,num_union,perc_loopsA_captured,perc_loopsB_captured
25,H9_D10,cortical,712533,65123,5264,772392,0.007388,0.080832
27,H9_D10,hippocampal,712533,98096,8138,802491,0.011421,0.082960
26,H9_D10,astrocyte,712533,72427,6463,778497,0.009070,0.089235
17,SL_D0,cortical,1187599,65123,6344,1246378,0.005342,0.097416
24,H9_D10,motor,712533,59650,5988,766195,0.008404,0.100386
19,SL_D0,hippocampal,1187599,98096,9864,1275831,0.008306,0.100555
13,H9_D0,cortical,1318716,65123,6768,1377071,0.005132,0.103926
33,H9_D28,cortical,1256959,65123,6797,1315285,0.005407,0.104372
18,SL_D0,astrocyte,1187599,72427,7795,1252231,0.006564,0.107626
15,H9_D0,hippocampal,1318716,98096,10584,1406228,0.008026,0.107894


In [70]:
loop_dfA.merge(loop_dfB, how='outer',on=['source','target']).shape

(1354446, 4)